# Pre-processing the Inter-beat Intervals
When you import your detected interbeat intervals along with their timestamps, they need to undergo a few preprocessing steps before they are ready for actual analysis. When you want to compare two devices (a criterion against a given wearable) and judge how well they perform for HR and HRV comparison, you need to realize there are many obstacles along the way that need to be taken care of beforehand. Sometimes the devices are not aligned and there is a lag, sometimes the length of a recording is shorter or longer, sometimes there are movement artifacts, outliers, etc. To summrize them, you need to:

- Correcting the possible lag between devices
- Segmenting continuous data from each wearable into smaller segments based on your experimental events
- Visually inspecting your signal: cropping the sily if necessary, adjusting possible lags in every conditiery useful when dealing with non-linearity in terms of lag
- Detecting possible outliers and ectopic beats
- Deciding whether your outlier detection and preprocessing indeed meaningfully cleaned the data

This notebook showcases how you can easily perform all these with `wearablehrv`.

<div style="border:1px solid; padding:10px; border-radius:5px; margin:10px 0;">

**Note**: Throughout the example notebooks and also in the code, we used the term "<u>criterion</u>," which refers to the device that the rest of the devices are compared against. This is also referred to as "reference system," "ground truth," and "gold standard" in the literature. This is usually an electrocardiography (ECG) device.

</div>

## Previous Steps

If you have not done so, first take a look at the following notebooks:

- [How to prepare your data for the individual pipeline](https://github.com/Aminsinichi/wearable-hrv/blob/master/docs/examples/individual_pipeline/1.individual_data_preparation.ipynb)


In [ ]:
# Importing Module
import wearablehrv

The code in the following cell has been explained in the previous notebook. Run it, so we can continue with the examples in this notebook.

In [ ]:
path = wearablehrv.data.download_data_and_get_path()
pp = "test" 
conditions = ['sitting', 'arithmetic', 'recovery', 'standing', 'breathing', 'neurotask', 'walking', 'biking'] 
devices = ["kyto", "heartmath", "rhythm", "empatica", "vu"]  
criterion = "vu" 
events = wearablehrv.individual.define_events (path, pp, conditions, already_saved= True, save_as_csv= False)
data = wearablehrv.individual.import_data (path, pp, devices)

print (events)
print (data)

## Correcting the (Linear) Lag

When you record simultaneously from different wearables, there is usually a lag present between your device and the criterion. Lag correction is important because you want your data to be matched up so you can compare them efficiently.

This can be consistent (linear) or inconsistent (non-linear). For the consistent lag, you can run the `lag_correction` function.It helps you visualize the entire recording of your selected device against the criterion device. In the `Start Time` and `End Time` widgets, it automatically selects a portion of the data for you to create the plot. Feel free to change this until you find a portion of the data where you can clearly see the criterion and your selected device. Then, with the lag slider (with millisecond precision), modify the lag, and once you are happy with it, click on `Save Lag`. This will shift the timestamps based on your selected lag, making it more accurate when chopping the data, so that the devices are synchronized. If you have multiple devices, you can easily switch between them with the `Device` widget until you align the data from all devices with the criterion.


You will see how you can deal with the non-linear lags later on.

In [ ]:
wearablehrv.individual.lag_correction (data, devices, criterion)

## Segmenting Your Continuous Data

After correcting the lag, you can now segment the continuous data into experimental conditions. This will be done using the `events` variable that you defined earlier. Simply call the `chop_data` function, and it will perform this task for all the devices in one go for you.

In [ ]:
data_chopped = wearablehrv.individual.chop_data (data, conditions, events, devices)

`data_chopped` is now a dictionary that contains data for each condition and each device. For instance, you can retrieve timestamps and interbeat intervals for the "Empatica" device by running the following code:

In [ ]:
data_chopped["empatica"]["sitting"]

## Visual Inspection, (Non-linear) Lag Correction, and Trimming

One of the powerful tools that `wearableHRV` offers is the `visual_inspection` function. By calling it, you will be able to see the interbeat intervals for each device in each condition, plotted against the criterion device, and easily switch between the conditions and devices.

It is not just a visualization tool; you can indeed modify your data in two ways here:

- Correct the inconsistent (non-linear) lag if it exists: You can now switch between the experimental conditions for each device and see if you find a sign of non-linear lag (e.g., if in the later conditions there is a lag, but no lag in the first conditions). You can easily adjust this with the Lag slider. The precision can be set to seconds or milliseconds with the `Precision` widget. After making changes, click on the `Save Lag` button to apply it to the data.
- Crop the beginning or end of the criterion or a selected device, independently, if necessary. This can come in handy if, for instance, you see there is a considerable difference in one of your conditions, and you want your signals from both devices to overlap as much as possible for comparison. This can be done easily using the dedicated widgets. Once you want to apply it to your data, click `Save Crop`.

A recommended approach is to select a condition, inspect it across all devices, align the lags as necessary, and then move on to the next condition. Repeat this process until all conditions have been reviewed. After finishing the initial pass, it's a good practice to revisit and verify the alignments once more.

Given you are potentially going to modify your data (e.g., trim it), it is good to keep track of the number of detected interbeat intervals before and after running the `visual_inspection` function. There is a dedicated function for this, called `calculate_ibi`, that keeps track of the detected interbeat intervals. **Please make sure to run them.**

In [ ]:
# Count how many interbeat intervals were detected in each device, for each condition, before trimming the data
nibi_before_cropping = wearablehrv.individual.calculate_ibi (data_chopped, devices, conditions)

In [ ]:
wearablehrv.individual.visual_inspection (data_chopped, devices, conditions,criterion)

In [ ]:
# Count how many interbeat intervals were detected in each device, for each condition, after trimming the data
nibi_after_cropping = wearablehrv.individual.calculate_ibi (data_chopped, devices, conditions)

### CheckPoint

Up to this point, we have made many time-consuming modifications to our dataset, and it is such a bummer to lose it! Therefore, it is advisable to create a backup so that you can retrieve the version of the `data_chopped` up to this point. The `save_backup` function saves the `data_chopped` variable in `.pkl` format at your specified path location. In order to retrieve this file, you can call the `import_backup` function.

In [ ]:
wearablehrv.individual.save_backup (pp, path, data_chopped) #To save the data_chopped 

In [ ]:
data_chopped = wearablehrv.individual.import_backup (pp, path) #To read the data_chopped again

## Removing Outliers and Ectopic Beats

Another essential component of preprocessing is to remove possible outliers and ectopic beats in every device and condition. Sometimes, there are giant movement artifacts that can mess up the agreement between a device and your criterion, but these are easily removable.

A great way to do this is by comparing the interbeat intervals to one another or to a moving average before and after it, and removing the suspected ones that exceed a specific threshold, or just removing a beat if it is too big or too small, etc. After removing these potential outliers, we can linearly interpolate the removed values.

Thankfully, there is a great package already available to deal with this, offering many preprocessing options, so we don't need to reinvent the wheel here :) We use `hrvanalysis` Python module for, and can check out the documentation here: https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html. 

It is incorporated into wearablehrv by calling the `pre_processing` function, and will apply it to all devices and all conditions separately.

**Note**: the shape of the `data_chopped` variable changes and is converted to "rr intervals" only after executing this code. The reason for this change is that all subsequent analyses and steps will be performed solely on inter-beat intervals (in other words, we will drop the timestamp column from now on).

In [ ]:
data_pp, data_chopped = wearablehrv.individual.pre_processing (data_chopped, devices, conditions, method="karlsson", custom_removing_rule = 0.25, low_rri=300, high_rri=2000)

It is good to keep track of how many artifacts were detected in each condition, on each device, as it can in fact inform us about signal quality later on. There is a dedicated function for this, called `calculate_artifact`. **Please make sure to run it**.

In [ ]:
artefact = wearablehrv.individual.calculate_artefact (data_chopped, data_pp, devices, conditions)

## A diagnostic Plot: Plotting Pre-processed vs. Original Data

An advantage of `wearableHRV` is that it offers you some diagnostic tools to judge how well you are doing. One of them is the `comparison_plot` function, which plots the preprocessed versus original data by running the `ibi_comparison_plot` function. The bottom row always shows your criterion device, and the top row shows the selected device.

This is useful because you can tell if the preprocessing approach was good enough, overcorrected the data, or did not perform well.A suggested approach is that you can experiment with the `pre_processing` function until you find the parameters that minimize the amount of detected artifact in your criterion device (because presumably it is an ECG, and cleaner compared to PPGs) and optimize the amount of artifact in your other devices. 

*Tip: If you have used `save_backup`, now you can retrieve your `data_chopped` by calling `import_backup`. Check out the Checkpoint mentioned a bit earlier.*

In [ ]:
wearablehrv.individual.ibi_comparison_plot (data_chopped, data_pp, devices, conditions, criterion, width=20 , height=10)

That's it! At this point, you should have been able to preprocess your data properly, such that you have taken care of all irrelevant components that may have played a role in minimizing the agreement between the device from a particular device and the criterion (such as lag, outliers, different length, etc).

## Next Steps

You're now ready to move on to the next notebook examples.

Continue by consulting:

- [Analyze your data](https://github.com/Aminsinichi/wearable-hrv/blob/master/docs/examples/individual_pipeline/3.individual_data_analysis.ipynb)